In [1]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D, Flatten, Dropout, Dense
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

Using TensorFlow backend.


In [2]:
model=Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        steps_per_epoch=10,
        validation_steps = 10,
        validation_data=test_set,

        )

model.save('mymodel.h5', model_saved)


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
10/10 [==============================] - 4s 373ms/step - loss: 0.7714 - acc: 0.5125 - val_loss: 0.6798 - val_acc: 0.5062
Epoch 2/10
10/10 [==============================] - 3s 257ms/step - loss: 0.6770 - acc: 0.5375 - val_loss: 0.6523 - val_acc: 0.5000
Epoch 3/10
10/10 [==============================] - 3s 251ms/step - loss: 0.6424 - acc: 0.6188 - val_loss: 0.5150 - val_acc: 0.7945
Epoch 4/10
10/10 [==============================] - 3s 272ms/step - loss: 0.4720 - acc: 0.7812 - val_loss: 0.3500 - val_acc: 0.8493
Epoch 5/10
10/10 [==============================] - 2s 234ms/step - loss: 0.3039 - acc: 0.8938 - val_loss: 0.2185 - val_acc: 0.9062
Epoch 6/10
10/10 [==============================] - 2s 235ms/step - loss: 0.2217 - acc: 0.9250 - val_loss: 0.2765 - val_acc: 0.8836
Epoch 7/10
10/10 [==============================] - 2s 239ms/step - loss: 0.1952 - acc: 0.9062 - val_loss: 0.2629 - val_acc: 

In [13]:
#To test for individual images
mymodel = load_model('mymodel.h5')
test_image = image.load_img(r'C:/Users/Gaurav/Documents/Machine Learning/FaceMaskDetector/test/without_mask/30.jpg',target_size = (150,150,3))
# test_image=image.load_img(r'C:/Users/Gaurav/Documents/Machine Learning/FaceMaskDetector/test/with_mask/1-with-mask.jpg', target_size = (150,150,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1.0

In [14]:
# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel = load_model('mymodel.h5')

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img = cap.read()
    face = face_cascade.detectMultiScale(img, scaleFactor = 1.1, minNeighbors = 4)
    for(x, y, w, h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg', face_img)
        test_image = image.load_img('temp.jpg', target_size=(150,150,3))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        pred = mymodel.predict(test_image)[0][0]
        if pred == 1:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
            cv2.putText(img, 'NO MASK', ((x + w)//2, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        else:
            cv2.rectangle(img, (x, y), (x + w,y + h), (0, 255, 0), 3)
            cv2.putText(img,'MASK', ((x + w)//2, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        datet =str(datetime.datetime.now())
        cv2.putText(img, datet, (400,450), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
          
    cv2.imshow('img', img)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()